In [1]:
import minsearch
import json
from tqdm.auto import tqdm


from openai import OpenAI
import tiktoken
from elasticsearch import Elasticsearch

from params import OPEN_API_KEY

#silence deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/Users/oksanamishchenko/.pyenv/versions/3.10.6/envs/llm_zoomcamp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Q1. Running Elastic


In [2]:
es_client = Elasticsearch('http://localhost:9200')
es_client

<Elasticsearch(['http://localhost:9200'])>

In [3]:
!curl localhost:9200

{
  "name" : "913c6d6250f3",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "5BMARHmNR5uZLTzaqfgo-w",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


Getting the data

In [4]:
import requests

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
documents[:3]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines 

### Q2. Indexing the data

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            #"section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "course-questions_h"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions_h'})

In [7]:
#es_client.indices.delete(index="course-questions_h")

In [8]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:04<00:00, 195.62it/s]


### Q3. Searching


In [14]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }#,
                # "filter": {
                #     "term": {
                #         "course": course
                #     }
                # }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        output = hit['_source']
        output['_score'] = hit['_score']
        result_docs.append(output)

    return result_docs

In [15]:
query = 'How do I execute a command in a running docker container?'

In [16]:
# What's the score for the top ranking result?
top_ranking = elastic_search(query)[0]
print(f'For the following top-ranking result:\n\n score is {top_ranking["_score"]}:\n question: {top_ranking["question"]} ')

For the following top-ranking result:

 score is 84.050095:
 question: How do I debug a docker container? 


### Q4. Filtering

In [17]:
def elastic_search(query, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                 "filter": {
                     "term": {
                         "course": course
                     }
                 }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        output = hit['_source']
        output['_score'] = hit['_score']
        result_docs.append(output)

    return result_docs

In [18]:
#Return 3 results. What's the 3rd question returned by the search engine?
res_ml = (elastic_search(query, "machine-learning-zoomcamp"))
res_ml[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

### Q5. Building a prompt

In [21]:
client = OpenAI(api_key=OPEN_API_KEY)

In [29]:
def build_prompt(question, search_results):
    """
    Build custom prompt based on the user query and retrieved data from knowledge base
    """

    # Building context from documents
    context = ""
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"


    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()
    prompt = prompt_template.format(question=question, context=context).strip()

    return prompt


In [39]:
prompt_output = build_prompt(question=query, search_results=res_ml)
prompt_output

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\n    Use only the facts from the CONTEXT when answering the QUESTION.\n\n    QUESTION: How do I execute a command in a running docker container?\n\n    CONTEXT:\n    Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/l

In [33]:
len_prompt = len(build_prompt(question=query, search_results=res_ml))
len_prompt

2730

### Q6. Tokens


In [35]:
# count tokens
#! pip install tiktoken

In [38]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [52]:
input_token_count = encoding.encode(prompt_output)
len(input_token_count)

625

In [46]:
def llm(prompt):
    """Sends generated prompt with context to OpenAI API"""
    response = client.chat.completions.create(
                model='gpt-4o',
                messages=[{'role':'user', 'content':prompt}]
    )

    return response.choices[0].message.content

### Generating ChatGPT answer

In [47]:
def rag(question, course):
    """Gets a question as input and outputs the ChatGpt answer with the giving context
    from knowledge data base"""
    search_results = elastic_search(question, course)
    prompt = build_prompt(question, search_results)
    response = llm(prompt)
    return response

In [48]:
gpt_answer = rag(query, "machine-learning-zoomcamp")
gpt_answer

'To execute a command in a running Docker container, you can use the `docker exec` command to open an interactive terminal within the specific container. Here are the steps to follow:\n\n1. Identify the container ID of the running container by using the `docker ps` command:\n   ```sh\n   docker ps\n   ```\n\n2. Use the `docker exec` command to open a shell in the container:\n   ```sh\n   docker exec -it <container-id> bash\n   ```\n\nReplace `<container-id>` with the actual container ID obtained from the `docker ps` command. This will allow you to execute commands within the running container interactively.'

### Calculating costs

In [56]:
output_token_count = encoding.encode(gpt_answer)
len(output_token_count)

134

In [57]:
# Costs:
input_costs = 0.005 * (len(input_token_count) / 1000)
input_costs

0.003125

In [66]:
output_costs = 0.015 * (len(output_token_count) / 1000)
output_costs

0.00201

In [68]:
sum_costs = round(input_costs + output_costs,3)
sum_costs

0.005